# Pre-Processing Pipeline for Pediatric GM Segmentation

Author: Ashwin Kumar  
Email: ashwin.kumar@vanderbilt.edu  
Description: Involves obtaining all spinal cord files, resampling the dimensions to 256 x 256 2D shaped slices,
automatically performing GM segmentation, and then manually editing the segmentation.

Note: The following sequence was run on Mac OS.




In [ ]:
import os
import shutil
import subprocess

In [ ]:
'''
Create New Directories
@input: base_dir for where folder needs to be created
@input: directory name for what needs to be created
'''
def create_directory(base_dir: str, dir_name: str):
    joined_path = os.path.join(base_dir, dir_name)
    if not os.path.isdir(joined_path):
        os.mkdir(joined_path)

### Getting all controls, create folder, copy the mffe_crop into directory

In [ ]:
patient_dir_folder = '/Users/captain/Documents/otherComputer/Documents/Patient_Data_Folder'
patient_dir_extension = 'DTI_Data/niftis'
working_dir = '/Users/captain/Documents/combined_roi/pediatric_gmseg'

# Go through each new line and copy the mFFE into the appropriate folder
with open(os.path.join(working_dir, 'Controls.txt')) as fp:
    Lines = fp.readlines()
    for line in Lines:
        dirID = str(int(line))
        create_directory(working_dir, dirID) # Create Directory
        os.chdir(os.path.join(patient_dir_folder,dirID, patient_dir_extension)) #Change to Patient_Dir
        shutil.copy2('mFFE_crop.nii', os.path.join(working_dir, dirID)) # Copy mFFE File
        

#### Processing the Lumbar Cord Files

In [ ]:
patient_dir_folder = '/Users/captain/Documents/otherComputer/Documents/Patient_Data_Folder'
patient_dir_extension = 'DTI_Data/niftis_Second'
working_dir = '/Users/captain/Documents/combined_roi/pediatric_gmseg'

# Repeat for the Lumbar cord files
with open(os.path.join(working_dir, 'Controls_second.txt')) as fp:
    Lines = fp.readlines()
    for line in Lines:
        dirID = str(int(line))
        create_directory(working_dir, dirID + '_second') # Create Directory
        os.chdir(os.path.join(patient_dir_folder,dirID, patient_dir_extension)) #Change to Patient_Dir
        shutil.copy2('mFFE_crop.nii', os.path.join(working_dir, dirID  + '_second')) # Copy mFFE File

### Resample mffe_crop.nii to 256 x 256

In [ ]:
# Go through every directory
for root, subdirectories, files in os.walk(working_dir):
    for subdirectory in subdirectories:
        if subdirectory.split('_')[0].isdigit():
            print(subdirectory)
            os.chdir(os.path.join(working_dir, subdirectory))
            # Resample mFFE_crop to 256 x 256
            subprocess.run(['sct_resample', '-i', 'mFFE_crop.nii', '-vox', '256x256x14', '-o', 'mFFE_crop_r.nii.gz'])

### Compute GM Seg using sct_deepseg_gm

In [ ]:
# Go through every directory
for root, subdirectories, files in os.walk(working_dir):
    for subdirectory in subdirectories:
        if subdirectory.split('_')[0].isdigit():
            print(subdirectory)
            os.chdir(os.path.join(working_dir, subdirectory))
            # Apply deepseg_gm
            subprocess.run(['sct_deepseg_gm', '-i', 'mFFE_crop_r.nii.gz'])

### Edit GMSeg

In [ ]:
# Go through every directory
for root, subdirectories, files in os.walk(working_dir):
    for subdirectory in subdirectories:
        if subdirectory.split('_')[0].isdigit():
            print(subdirectory)
            os.chdir(os.path.join(working_dir, subdirectory))
            # Edit GM Seg
            subprocess.run(['fsleyes', 'mFFE_crop_r.nii.gz', '-cm', 'greyscale', '-a', '100.0', 'mFFE_crop_r_gmseg.nii.gz', '-cm', 'red', '-a', '70.0'])